# Connect4-Zero Training

**Setup:** Runtime > Change runtime type > GPU (T4/A100/H100)

In [ ]:
# Clean clone and install
!rm -rf connect4-zero
!git clone https://github.com/mindswim/connect4-zero.git
%cd connect4-zero
!pip install -e . -q

import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

In [ ]:
# Quick benchmark
!c4z benchmark --sims 50 --games 3

In [ ]:
# Train with turbo config (fast iterations ~1-2 min each)
!c4z train --config configs/turbo.yaml --iterations 100

In [ ]:
# Test against random player
from connect4zero.net import load_checkpoint
from connect4zero.eval import Arena
from connect4zero.utils import get_device

device = get_device()
model, _ = load_checkpoint('checkpoints/best.pt', device)

arena = Arena(num_simulations=50, device=device)
result = arena.evaluate_vs_random(model, num_games=50)
print(f"\nWin rate vs random: {result.win_rate*100:.1f}%")
print(f"W:{result.wins} L:{result.losses} D:{result.draws}")

In [ ]:
# Optional: Save to Google Drive
from google.colab import drive
drive.mount('/content/drive')
!cp checkpoints/best.pt /content/drive/MyDrive/connect4-zero-best.pt
print("Saved to Google Drive")

In [ ]:
# Optional: Download checkpoint
from google.colab import files
files.download('checkpoints/best.pt')